# Avance #2 del Proyecto Integrador

### Objetivos

- **Conexión con la API de Yelp**: Integrar la API de Yelp para obtener información detallada sobre negocios locales en una ciudad seleccionada, basada en el país correspondiente.
  
- **Extracción de Datos**: Recopilar información relevante de cada negocio, incluyendo:
  - Nombre del negocio
  - Categoría
  - Calificación
  - Número de reseñas
  
- **Integración de Información**: Incorporar los datos obtenidos al análisis en curso sobre la ciudad/país específico.

- **Documentación de Resultados**: Documentar de manera clara y detallada los resultados obtenidos y los pasos seguidos durante el proceso.

---

#### Continuación del Proyecto

Para dar continuidad a este segundo avance del proyecto integrador, nos proponemos **obtener datos actualizados sobre restaurantes** en una ciudad específica. Tras el análisis realizado en el primer avance, hemos decidido centrar nuestra atención en la **ciudad de Chicago**. 

En este documento detallaremos el proceso completo, que va desde la **conexión a la API de Yelp** para obtener la información necesaria, hasta el **almacenamiento de los datos** generados, la **limpieza** de los mismos y el posterior **análisis** en busca de **insights valiosos**, con el fin de complementar los hallazgos obtenidos en la primera parte del proyecto.

---

### Carga de Librerías

Al igual que en la primera parte, procederemos a cargar todas las **librerías** necesarias para llevar a cabo el análisis y tratamiento de las anomalías que encontremos. Estas librerías serán esenciales para procesar, limpiar y visualizar los datos obtenidos, garantizando una interpretación precisa de los resultados.


In [47]:
# Carga de librerías necesarias

import pandas as pd # Importa la librería pandas, esencial para la manipulación y análisis de datos. 
                    # Se usa para trabajar con DataFrames, que son estructuras de datos tabulares 
                    # similares a las hojas de cálculo.

import numpy as np # Importa la librería numpy, fundamental para operaciones numéricas y matemáticas 
                   # de alto rendimiento. Se utiliza para trabajar con arrays y matrices multidimensionales.

from IPython.display import display # Importa la función 'display' del módulo IPython.display. 
                                    # Esta función es útil en entornos como Jupyter Notebook para 
                                    # mostrar de manera clara DataFrames y otros objetos.

from mega_funcion import imputar # Importa la función 'imputar' desde un archivo local llamado 
                                 # 'mega_funcion.py'. Esto sugiere que 'imputar' es una función personalizada 
                                 # para manejar valores faltantes en los datos.

from fuzzywuzzy import fuzz # Importa el módulo 'fuzz' de la librería fuzzywuzzy. 
                            # Se usa para la comparación "difusa" de cadenas de texto, lo que 
                            # permite encontrar similitudes entre textos que no son idénticos, 
                            # ideal para limpieza de datos con errores tipográficos.

import requests # Importa la librería requests, estándar para hacer peticiones HTTP. 
                # Se utiliza para interactuar con APIs y obtener datos de la web.

import ast # Importa el módulo 'ast' (Abstract Syntax Trees). Se usa para evaluar y convertir 
           # cadenas de texto que representan estructuras de datos (como diccionarios o listas) 
           # en objetos de Python reales, evitando problemas de seguridad que podrían surgir con 'eval()'.

### Conexión a la API de Yelp para Obtener Datos de Restaurantes en Chicago

Una vez cargadas nuestras librerías, procedemos con los pasos necesarios para **conectarnos a la API de Yelp** y obtener los datos relevantes sobre los **restaurantes en la ciudad de Chicago**.

#### Disclaimer:
Debido a las limitaciones inherentes a la API de Yelp, **solo se nos permite consultar hasta 200 registros** de restaurantes al realizar búsquedas por ciudad (sin modificar los parámetros de búsqueda). Por esta razón, hemos decidido continuar con esta cantidad de registros en el presente análisis.

No obstante, cabe destacar que es posible obtener más datos utilizando un enfoque diferente. El código necesario para ello está disponible en un archivo separado que se puede encontrar en la **carpeta del proyecto**. 

Este archivo incluye la implementación necesaria para superar la limitación de 200 registros y continuar con la recopilación de datos de forma más extensa.


In [48]:
# Definición de las variables para autenticarse y acceder a la API de Yelp

# URL de la API de búsqueda de negocios de Yelp.
# Esta es la dirección a la que tu código enviará las solicitudes para obtener información.
url = "https://api.yelp.com/v3/businesses/search" 

# ID del cliente para la API de Yelp.
# Este es un identificador único que te asigna Yelp para rastrear tus solicitudes.
cliente_id = 'GWOCZh9-BmZxtdsAjr7Gug'

# Clave de la API de Yelp (API Key).
# Es tu "contraseña" de acceso. Se usa para autenticarte y verificar que tienes permiso para
# acceder a los datos de la API. Mantén esta clave confidencial.
api_key = 'FHVvXoNmTXIl9DuxYis7AV5uLPujm9MLwrhgs5NgvCfaOxd3V6mxt6dQU8eEqYJiGxe816XATx7ufWjbMWqbV-2Uku1jxBJv8BGRC74NroLPl27PDQqs0tDixit-YHYx'

In [49]:
# Encabezados de la solicitud HTTP.
# Esta es la parte de la solicitud que contiene la información de autenticación.
# El valor 'Authorization' le dice a la API quién eres usando tu clave de API (api_key)
# y el método de autenticación 'Bearer'.
headers = {'Authorization': 'Bearer %s' % api_key}

# Variable para definir la ubicación de la búsqueda.
# Esto hace que sea fácil cambiar la ciudad sin tener que editar los parámetros de la solicitud.
ciudad = 'Chicago'

# Diccionario de parámetros para la búsqueda.
# Estos son los filtros que le dicen a la API qué tipo de negocios y cuántos quieres.
params = {
    # Busca negocios que coincidan con el término 'restaurants'
    'term': 'restaurants', 
    
    # Define la ubicación de la búsqueda usando la variable 'ciudad'
    'location': ciudad, 
    
    # Limita el número de resultados a un máximo de 50 por solicitud
    'limit': 50
}

In [50]:
# Realiza una solicitud HTTP de tipo GET a la API de Yelp.
# Los datos de la solicitud son construidos usando las variables que definiste anteriormente.
response = requests.get(url=url, params=params, headers=headers)

# Muestra el objeto de respuesta de la solicitud.
# Esto te permite ver el estado de la conexión. Un código 200 significa que la solicitud
# fue exitosa y la API respondió correctamente. Otros códigos (ej. 404, 401) indican un problema.
response


<Response [200]>

### Ciclo para Obtener Hasta 1000 Resultados de Yelp y Guardarlos en CSV

Como se especificó anteriormente, la API de Yelp permite obtener **hasta 200 resultados** por consulta, con un límite de **50 resultados por solicitud**. Por lo tanto, realizaremos **4 solicitudes** para obtener un total de **1000 resultados**. El ciclo se detiene automáticamente si se obtienen menos de 50 resultados, indicando que no hay más datos disponibles.

A continuación, se describe el proceso de obtención y almacenamiento de los resultados:

1. **Inicialización de la Lista de Resultados**: 
   - Se crea una lista vacía `all_results` para almacenar los resultados de cada solicitud.

2. **Ciclo de Solicitudes**: 
   - Se itera con un desplazamiento (`offset`) de 50 en 50, hasta alcanzar los 1000 resultados. En cada iteración:
     - **Parámetros de la Solicitud**: Se actualizan los parámetros para incluir `term='restaurants'`, `location='ciudad'`, `limit=50` y el `offset` correspondiente.
     - **Realización de la Solicitud**: Se envía una solicitud `GET` a la API de Yelp con los parámetros y las cabeceras adecuadas.
     - **Almacenamiento de Resultados**: Se obtiene la respuesta en formato JSON y se extraen los negocios de la clave `businesses`. Estos resultados se agregan a la lista `all_results`.

3. **Condición de Parada**: 
   - Si en cualquier solicitud se obtienen menos de 50 resultados, el ciclo se detiene, ya que no hay más datos disponibles.

4. **Normalización y Guardado en CSV**: 
   - Una vez completado el ciclo, los datos se normalizan utilizando `pandas` para convertirlos en un DataFrame.
   - Finalmente, se guarda el DataFrame en un archivo CSV llamado `chicago_restaurants.csv`, asegurando que los datos estén almacenados localmente para su posterior análisis.

Este proceso asegura que se obtengan hasta **1000 resultados** sobre restaurantes en Chicago, distribuidos en **4 solicitudes** de 50 registros cada una, y se almacenen de manera eficiente en un archivo CSV.


In [51]:
# Lista para almacenar todos los datos de los restaurantes
all_results = [] 

# Ciclo para hacer múltiples solicitudes a la API, paginando los resultados
# El 'offset' aumenta de 50 en 50, permitiendo obtener hasta 1000 resultados en total
for offset in range(0, 1000, 50): 
    
    # Se crea un nuevo diccionario de parámetros para cada solicitud
    # Se incluye el 'offset' actual para desplazar la búsqueda
    params = {'term': 'restaurants', 'location': ciudad, 'limit': 50, 'offset': offset}
    
    # Se realiza la solicitud GET a la API con los parámetros y encabezados definidos
    response = requests.get(url, params=params, headers=headers)
    
    # Se convierte la respuesta de texto JSON a un diccionario de Python
    data = response.json()
    
    # Se extrae la lista de negocios ('businesses') de los datos y se añaden a 'all_results'
    # .get('businesses', []) evita errores si la llave no existe, devolviendo una lista vacía
    all_results.extend(data.get('businesses', []))
    
    # Lógica de "break" para detener el ciclo antes de llegar al límite de 1000
    # Si la cantidad de resultados de la última solicitud es menor a 50,
    # significa que se ha llegado al final de los datos disponibles
    if len(data.get('businesses', [])) < 50:
        break

# --- Procesamiento y Guardado de los Datos ---

# Se normalizan los datos de la lista de diccionarios a un DataFrame de Pandas
# Esta función es ideal para convertir datos JSON anidados en un formato de tabla plano
df = pd.json_normalize(all_results)

# Se guarda el DataFrame en un archivo CSV
# 'chicago_restaurants.csv' es el nombre del archivo de salida
# 'index=False' evita que Pandas escriba el índice del DataFrame como una columna más en el CSV
df.to_csv('chicago_restaurants.csv', index=False)

### Exploración Inicial de los Datos

Una vez que hemos obtenido los resultados de las solicitudes a la API de Yelp y los hemos convertido en un archivo `.csv`, es momento de iniciar con la **exploración inicial** de los datos. Este paso es fundamental para identificar los **principales problemas** a resolver y entender mejor la estructura de la información obtenida.

Durante esta fase, realizaremos una serie de análisis preliminares para:
- Verificar la **completitud** de los datos.
- Identificar posibles **valores faltantes** o **errores** en los registros.
- Analizar la **distribución** de las variables clave, como la **calificación**, **número de reseñas** y **categorías** de los restaurantes.
- Identificar **anomalías** o **valores atípicos** que puedan afectar la calidad del análisis posterior.

Con base en esta exploración, podremos definir las acciones necesarias para **limpiar** y **transformar** los datos antes de realizar análisis más detallados.


In [52]:
# Carga de datos

# Lee el archivo 'chicago_restaurants.csv' y lo carga en un DataFrame de pandas.
# Esto te permite trabajar con los datos que guardaste de la API, sin tener que
# volver a hacer las solicitudes en cada ejecución.
df_original = pd.read_csv('chicago_restaurants.csv') 

# Creación de una copia de trabajo

# Crea una copia del DataFrame original.
# Se usa .copy() para asegurarse de que cualquier cambio que hagas en 'df_res'
# no afecte a 'df_original', manteniendo una versión limpia de tus datos.
df_res = df_original.copy()

### Forma del DataFrame

Como primer paso en la exploración inicial, verificamos la **forma del DataFrame** obtenido a partir de los datos de la API de Yelp.  
Este procedimiento nos permite conocer la **cantidad de filas y columnas**, es decir, cuántos registros de restaurantes fueron descargados y cuántas variables tenemos disponibles para el análisis.

Este paso es clave porque:
- Confirma que la carga de datos fue exitosa.
- Nos ayuda a dimensionar el tamaño del conjunto de datos.
- Facilita la planeación de los siguientes procesos de **limpieza**, **transformación** y **análisis exploratorio**.


In [53]:
# El atributo .shape de un DataFrame devuelve una tupla con sus dimensiones (filas, columnas).
# Por ejemplo, para un DataFrame de 100 filas y 5 columnas, .shape devolvería (100, 5).

# Usamos una "f-string" para insertar los valores del .shape en un mensaje de texto.
# df_res.shape[0] accede al primer elemento de la tupla, que es el número de filas.
# df_res.shape[1] accede al segundo elemento de la tupla, que es el número de columnas.

print(f"El DataFrame tiene {df_res.shape[0]} filas y {df_res.shape[1]} columnas.")

El DataFrame tiene 200 filas y 24 columnas.


### Visualización de las Primeras 10 Filas

Para obtener un **vistazo rápido** a la información y comprender cómo están organizados los datos, mostramos las **primeras 10 filas** del DataFrame. Esto es importante para comenzar con el **análisis exploratorio preliminar** y conocer la **estructura** de los datos, especialmente en términos de las **columnas** y cómo están clasificadas.

Este paso nos permite:
- Observar el **contenido** y formato de las primeras filas del conjunto de datos.
- Validar la **presencia de columnas clave** como el nombre del restaurante, calificación, número de reseñas, etc.
- Comenzar a identificar **posibles inconsistencias** o **valores faltantes** en las columnas.

Es una forma inicial de asegurarnos de que los datos se cargaron correctamente y entender mejor las variables que tendremos que analizar.


In [54]:
df_res.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],...,41.884193,-87.647946,809 W Randolph,NaN,NaN,Chicago,60607,US,IL,"['809 W Randolph', 'Chicago, IL 60607']"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",...,41.890694,-87.624782,444 N Michigan Ave,NaN,NaN,Chicago,60611,US,IL,"['444 N Michigan Ave', 'Chicago, IL 60611']"
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",...,41.917275,-87.698577,2833 W Armitage Ave,NaN,NaN,Chicago,60647,US,IL,"['2833 W Armitage Ave', 'Chicago, IL 60647']"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],...,41.881689,-87.625006,12 S Michigan Ave,NaN,NaN,Chicago,60603,US,IL,"['12 S Michigan Ave', 'Chicago, IL 60603']"
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],...,41.889390,-87.635240,226 W Kinzie,NaN,NaN,Chicago,60654,US,IL,"['226 W Kinzie', 'Chicago, IL 60654']"
5,cgIuo3geaxw1Sfhbqm5qTA,alla-vita-chicago-4,Alla Vita,https://s3-media0.fl.yelpcdn.com/bphoto/X4XMxK...,False,https://www.yelp.com/biz/alla-vita-chicago-4?a...,740,"[{'alias': 'italian', 'title': 'Italian'}]",4.4,"['pickup', 'delivery']",...,41.884680,-87.642322,564 W Randolph St,NaN,NaN,Chicago,60661,US,IL,"['564 W Randolph St', 'Chicago, IL 60661']"
6,9nUrDQ_REhU6sgcgXFAyfA,aba-chicago-2,Aba,https://s3-media0.fl.yelpcdn.com/bphoto/CeQacW...,False,https://www.yelp.com/biz/aba-chicago-2?adjust_...,1857,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.4,"['pickup', 'delivery']",...,41.886944,-87.648795,302 N Green St,Fl 3,NaN,Chicago,60607,US,IL,"['302 N Green St', 'Fl 3', 'Chicago, IL 60607']"
7,PZe0q_153VHUnaR-8dOTJg,the-dearborn-chicago-2,The Dearborn,https://s3-media0.fl.yelpcdn.com/bphoto/eSXeGi...,False,https://www.yelp.com/biz/the-dearborn-chicago-...,2605,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,"['pickup', 'delivery']",...,41.884253,-87.629315,145 N Dearborn St,NaN,NaN,Chicago,60602,US,IL,"['145 N Dearborn St', 'Chicago, IL 60602']"
8,VPJk-SEWSWS_nGoQvM-COw,penumbra-chicago,Penumbra,https://s3-media0.fl.yelpcdn.com/bphoto/IlSPQK...,False,https://www.yelp.com/biz/penumbra-chicago?adju...,995,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.8,"['restaurant_reservation', 'delivery']",...,41.924459,-87.710898,3309 W Fullerton Ave,NaN,NaN,Chicago,60647,US,IL,"['3309 W Fullerton Ave', 'Chicago, IL 60647']"
9,ZXsqdTNHSqxBxCXcbmOiLA,bigsuda-chicago,Bigsuda,https://s3-media0.fl.yelpcdn.com/bphoto/Yb2j5v...,False,https://www.yelp.com/biz/bigsuda-chicago?adjus...,44,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,[],...,41.906445,-87.671742,1362 N Milwaukee Ave,NaN,NaN,Chicago,60622,US,IL,"['1362 N Milwaukee Ave', 'Chicago, IL 60622']"


### Análisis de las Columnas del DataFrame

Para entender mejor la estructura de nuestros datos, realizamos un pequeño ciclo que nos permite conocer tanto el **nombre** como la **cantidad total de columnas** en el DataFrame. Este paso es crucial para confirmar qué variables están disponibles para el análisis y asegurar que los datos estén correctamente organizados.

Este ciclo nos permite:
- **Verificar** la cantidad exacta de columnas presentes en el DataFrame.
- **Listar** los nombres de todas las columnas para asegurarnos de que no falta ninguna variable clave.
- **Identificar posibles inconsistencias** en los nombres de las columnas, como espacios o caracteres no deseados.

Al realizar este análisis, nos aseguramos de que los datos estén listos para las siguientes etapas del proceso de exploración y análisis.


In [55]:
# Itera sobre los nombres de todas las columnas en el DataFrame 'df_res'.
# La función 'enumerate' asigna un índice a cada columna, empezando desde 1.
for i, column in enumerate(df_res.columns, 1):
    # Imprime el índice (i) y el nombre de la columna (column)
    print(f"{i} : {column}")

1 : id
2 : alias
3 : name
4 : image_url
5 : is_closed
6 : url
7 : review_count
8 : categories
9 : rating
10 : transactions
11 : price
12 : phone
13 : display_phone
14 : distance
15 : coordinates.latitude
16 : coordinates.longitude
17 : location.address1
18 : location.address2
19 : location.address3
20 : location.city
21 : location.zip_code
22 : location.country
23 : location.state
24 : location.display_address


### Identificación de Columnas Numéricas y Categóricas

Es fundamental conocer cuáles son las **columnas numéricas** y **categóricas** de nuestro DataFrame, ya que esto nos permitirá anticipar el uso de las funciones y herramientas adecuadas, dependiendo de los tipos de datos que queramos manipular.

- **Columnas numéricas**: Son aquellas que contienen datos cuantitativos, como **calificaciones**, **número de reseñas** y cualquier otro valor que se pueda medir y operar matemáticamente.
- **Columnas categóricas**: Son aquellas que contienen datos cualitativos, como **nombre de los restaurantes**, **categoría de negocio** o **ubicación**, que no se pueden operar directamente con funciones matemáticas.

Conocer la clasificación de las columnas es crucial para:
- **Seleccionar las funciones adecuadas** para limpiar, transformar o analizar los datos, como operaciones estadísticas para variables numéricas o conteos de frecuencia para variables categóricas.
- **Evitar errores** al aplicar funciones inapropiadas a las columnas.
- **Optimizar el análisis exploratorio**, ya que nos permite elegir las herramientas correctas para cada tipo de dato.

Este paso garantiza que estemos listos para trabajar con los datos de manera eficiente y con precisión.


In [56]:
# --- Identificación de Columnas por Tipo de Dato ---

# Identificar columnas categóricas (texto)
# Usa .select_dtypes() para filtrar las columnas por su tipo de dato.
# 'object' en pandas generalmente se refiere a cadenas de texto (strings).
columnas_categoricas = df_res.select_dtypes(include='object').columns.tolist() 

# Imprime la lista de nombres de columnas categóricas
print("Columnas categóricas:", columnas_categoricas)

# Imprime el número total de columnas identificadas
print("Número de columnas categóricas:", len(columnas_categoricas))

# Identificar columnas numéricas
# De nuevo, usa .select_dtypes(), pero esta vez con 'number'
# 'number' incluye todos los tipos de datos numéricos (int, float, etc.).
columnas_numericas = df_res.select_dtypes(include='number').columns.tolist() 

# Imprime la lista de nombres de columnas numéricas
print("Columnas numéricas:", columnas_numericas)

# Imprime el número total de columnas identificadas
print("Número de columnas numéricas:", len(columnas_numericas))

Columnas categóricas: ['id', 'alias', 'name', 'image_url', 'url', 'categories', 'transactions', 'price', 'display_phone', 'location.address1', 'location.address2', 'location.address3', 'location.city', 'location.country', 'location.state', 'location.display_address']
Número de columnas categóricas: 16
Columnas numéricas: ['review_count', 'rating', 'phone', 'distance', 'coordinates.latitude', 'coordinates.longitude', 'location.zip_code']
Número de columnas numéricas: 7


### Análisis Técnico de los Datos para Limpieza y Normalización

Una vez que hemos identificado los datos generales y las categorías con las que estaremos trabajando, el siguiente paso es obtener información más **técnica** sobre los valores presentes en el DataFrame. Este análisis es esencial para determinar qué **categorías** o **valores** necesitan ser **limpiados** y **normalizados** antes de proceder con análisis más avanzados.

Este paso nos permite:
- **Detectar valores faltantes** o **nulos** en las columnas, lo que puede afectar la calidad de los análisis posteriores.
- **Identificar inconsistencias** o errores en los datos, como valores atípicos, duplicados o entradas mal formateadas.
- Evaluar la **distribución** de las categorías, asegurándonos de que las variables categóricas estén correctamente etiquetadas y que no haya errores tipográficos o inconsistencias en los nombres.
- Verificar la **normalización** de las columnas numéricas, asegurando que los valores estén en rangos apropiados y que no haya valores extremos no deseados.

Con esta información técnica, podemos planificar con precisión qué transformaciones son necesarias para que los datos sean adecuados para un análisis más detallado y fiable.


In [57]:
# Muestra un resumen conciso del DataFrame 'df_res'.
# Esta es una de las primeras y más importantes funciones para el análisis exploratorio de datos.
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        200 non-null    object 
 1   alias                     200 non-null    object 
 2   name                      200 non-null    object 
 3   image_url                 200 non-null    object 
 4   is_closed                 200 non-null    bool   
 5   url                       200 non-null    object 
 6   review_count              200 non-null    int64  
 7   categories                200 non-null    object 
 8   rating                    200 non-null    float64
 9   transactions              200 non-null    object 
 10  price                     134 non-null    object 
 11  phone                     190 non-null    float64
 12  display_phone             190 non-null    object 
 13  distance                  200 non-null    float64
 14  coordinate

### Análisis Detallado con Función Estadística de Pandas

Para obtener una visión más detallada de la información técnica y, especialmente, **estadística** de nuestros datos, utilizamos una función incluida en la librería **Pandas**. Esta función nos proporciona un resumen estadístico completo de las columnas numéricas, permitiéndonos detectar patrones y anomalías de manera eficiente.

Con esta herramienta, podemos obtener información clave como:
- **Promedio (mean)**: Para identificar la tendencia central de las variables numéricas.
- **Desviación estándar (std)**: Para entender la dispersión de los datos.
- **Mínimo y máximo**: Para identificar los valores extremos o atípicos.
- **Cuartiles**: Para evaluar la distribución de los datos y detectar posibles sesgos.

Este análisis estadístico es fundamental porque:
- **Facilita la detección de anomalías** o **valores atípicos** que podrían influir negativamente en el análisis posterior.
- **Permite realizar comparaciones por categoría** y detectar posibles **desviaciones** dentro de cada grupo.
- Ayuda a **planificar la limpieza de datos** y aplicar transformaciones como normalización o eliminación de valores erróneos.

De esta manera, tener una visión clara de las estadísticas básicas de nuestros datos nos permite tomar decisiones informadas sobre cómo proceder con la limpieza y preparación de los datos para el análisis.


In [58]:
# Genera un resumen estadístico del DataFrame 'df_res'.
# El argumento 'include='all'' le indica a pandas que debe incluir
# tanto las columnas numéricas como las categóricas en el resumen.
df_res.describe(include='all')

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
count,200,200,200,200,200,200,200.000000,200,200.000000,200,...,200.000000,200.000000,200,10,1,200,200.000000,200,200,200
unique,200,200,200,200,1,200,NaN,173,NaN,12,...,NaN,NaN,197,9,1,3,NaN,1,1,197
top,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,NaN,"[{'alias': 'italian', 'title': 'Italian'}]",NaN,"['pickup', 'delivery']",...,NaN,NaN,59 W Hubbard St,Lower Level,Willis Tower,Chicago,NaN,US,IL,"['59 W Hubbard St', 'Chicago, IL 60654']"
freq,1,1,1,1,200,1,NaN,9,NaN,84,...,NaN,NaN,2,2,1,198,NaN,200,200,2
mean,NaN,NaN,NaN,NaN,NaN,NaN,876.090000,NaN,4.401500,NaN,...,41.905545,-87.657499,NaN,NaN,NaN,NaN,60624.730000,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,1620.164432,NaN,0.231768,NaN,...,0.026011,0.028987,NaN,NaN,NaN,NaN,29.663051,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,3.800000,NaN,...,41.848943,-87.802095,NaN,NaN,NaN,NaN,60302.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,101.750000,NaN,4.300000,NaN,...,41.888774,-87.677499,NaN,NaN,NaN,NaN,60611.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,307.000000,NaN,4.400000,NaN,...,41.900134,-87.650453,NaN,NaN,NaN,NaN,60622.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,838.250000,NaN,4.500000,NaN,...,41.920080,-87.632185,NaN,NaN,NaN,NaN,60647.000000,NaN,NaN,NaN


---------------------

### Exploración Inicial del DataFrame de Yelp para Restaurantes en Chicago

Tras realizar la exploración inicial del DataFrame obtenido de la API de Yelp para restaurantes en Chicago, se identificaron los siguientes puntos clave:

- **Tamaño del DataFrame**: El DataFrame cuenta con **200 filas** y **24 columnas**, lo que representa una muestra manejable pero suficiente para análisis exploratorios y posteriores modelados.
  
- **Tipos de columnas**: Se identificaron **16 columnas categóricas** y **7 columnas numéricas**, lo que permite un análisis tanto descriptivo como inferencial de los datos.

- **Valores nulos**: Se observaron **valores nulos** en varias columnas, siendo especialmente relevantes en `price`, `phone`, y `image_url`. La columna `price` presentaba cerca del **35% de valores faltantes**, lo que podría afectar la calidad de los análisis si no se trata adecuadamente.

- **Columnas redundantes**: Existen columnas redundantes o con información duplicada, como las diferentes variantes de dirección (`location.address1`, `location.address2`, `location.address3`, `location.display_address`) y teléfono (`phone`, `display_phone`), lo que sugiere la necesidad de unificación para evitar inconsistencias.

- **Tipos de datos incorrectos**: Se detectaron **tipos de datos incorrectos** en algunas columnas. Por ejemplo, `phone` estaba en formato numérico cuando debería ser texto, y columnas como `categories` y `transactions` estaban en formato string representando listas o diccionarios.

- **Columnas irrelevantes**: Algunas columnas, como `distance` e `is_closed`, no aportan información relevante para el análisis y pueden eliminarse para simplificar el DataFrame.

- **Transformación de datos**: Se observó que la columna `price` utiliza símbolos (`$`, `$$`, etc.) para representar rangos de precios, lo que requiere **transformación a valores numéricos** para facilitar el análisis.

- **Transformación de categorías**: La columna `categories` contiene información valiosa sobre el tipo de restaurante, pero requiere **transformación** para ser utilizada de manera eficiente en análisis posteriores.

- **Imputación de valores faltantes**: Finalmente, se detectó la necesidad de **imputar valores faltantes** de manera inteligente, especialmente en `price`, para no perder información relevante y evitar sesgos en los resultados.

### Resumen

La exploración inicial permitió identificar los **principales retos de limpieza y transformación de los datos**, así como las oportunidades para enriquecer el análisis posterior. El siguiente paso será abordar estos puntos mediante **técnicas de limpieza**, **imputación** y **transformación de variables**.

----------------------------


## 2. Limpieza de Datos 

#### Identificación de Valores Nulos, Duplicados y Errores

Una vez realizado el análisis inicial para detectar las principales anomalías en el DataFrame, comenzamos el proceso de **limpieza de datos**. El objetivo de esta etapa es corregir los errores y preparar el DataFrame para futuros análisis, asegurando que los datos sean consistentes y fiables.

En esta sección se llevarán a cabo las siguientes acciones:

- **Resolver valores nulos y fuera de rango**: Se imputarán los valores faltantes tomando en cuenta las categorías relacionadas, para evitar que los promedios generales se vean alterados por datos faltantes o erróneos.
  
- **Identificación de valores duplicados**: Se buscarán registros duplicados en el DataFrame para decidir la mejor estrategia para manejarlos, ya sea sustituyéndolos, eliminándolos o simplemente creando una advertencia.
  
- **Cambio de tipo de variables**: Se cambiará el tipo de las columnas que no correspondan con su tipo de dato correcto, para evitar inconsistencias y errores en los análisis posteriores.
  
- **Unión de columnas irrelevantes**: Se combinarán aquellas columnas que están separadas pero que contienen información redundante o irrelevante.

---------------------------

### Comenzamos con la identificación de valores nulos

El primer paso de esta etapa de limpieza es **identificar los valores nulos**. Comenzamos con la columna `id`, ya que, aunque en la fase anterior se verificó que no había valores nulos, es importante realizar una **segunda verificación** para asegurarnos de que no se haya pasado por alto ningún dato faltante en esta columna clave.



In [59]:
# Identifica las filas donde el valor en la columna 'id' está duplicado.
# 1. df_res['id']: Selecciona la columna 'id' del DataFrame.
# 2. .duplicated(): Este método devuelve una serie booleana (True/False)
#    donde True indica que el valor ya ha aparecido antes.
# 3. df_res['id'][...]: Usa la serie booleana para filtrar la columna 'id'
#    y mostrar solo los valores que son True (es decir, los duplicados).
id_ducplicado = df_res['id'][df_res['id'].duplicated()]

# Muestra la serie de los IDs duplicados encontrados.
id_ducplicado

Series([], Name: id, dtype: object)

### Identificación de Valores Únicos en la Columna 'location.city'

Ahora que hemos confirmado que no existen valores nulos en el DataFrame, procedemos a **buscar los valores únicos** por columna en `location.city`. Esto se hace porque notamos que uno de los valores era diferente al resto, lo que despertó nuestra curiosidad.

Al realizar esta verificación, descubrimos que, aunque uno de los registros tenía un nombre distinto, todos los valores seguían haciendo referencia a la **misma ciudad**. Este hallazgo nos llevó a la conclusión de que, si bien el nombre parecía diferente, no afectaba la consistencia de los datos, ya que todos los registros correspondían a la misma ubicación geográfica.

Este tipo de validación es útil para asegurarnos de que los datos categóricos estén correctamente uniformados y no presenten inconsistencias, especialmente cuando se tratan de nombres o etiquetas que podrían tener variantes o errores tipográficos.


In [60]:
# Cuenta la frecuencia de los valores únicos en una columna específica

# 1. df_res['location.city']: Selecciona la columna 'location.city' del DataFrame.
# 2. .value_counts(): Este método de pandas cuenta cuántas veces aparece cada valor único
#    en la columna seleccionada. El resultado es una Serie que muestra cada valor único
#    y su respectiva frecuencia.
valores_unicos = df_res['location.city'].value_counts()
 
# Imprime un título para los resultados
print("Valores únicos por columna:")

# Imprime la Serie con los valores únicos y sus conteos
print(valores_unicos)

Valores únicos por columna:
location.city
Chicago         198
Lincoln Park      1
Oak Park          1
Name: count, dtype: int64


### Unificación de Columnas de Dirección

Después de realizar el análisis en la primera fase, nos percatamos de la **cantidad de columnas redundantes** en el DataFrame. Esto se refiere a aquellas columnas en las que la información se repite innecesariamente, lo que puede dificultar el análisis y la manipulación de los datos.

En particular, las columnas **'location.address1'**, **'location.address2'** y **'location.address3'** contenían diferentes partes de la misma dirección. Tras evaluar su contenido, decidimos unificar estas columnas en una sola, ya que consideramos que es más práctico trabajar con una única columna de dirección en lugar de múltiples campos separados. Las columnas adicionales contenían detalles de la dirección que, en realidad, no aportaban valor adicional cuando se mantenían por separado.

- **Acción realizada**: Eliminamos las 3 columnas originales y creamos una nueva columna con la dirección unificada.
- **Posibilidad futura**: Si en algún momento necesitamos los datos de la dirección de forma separada, podemos volver a dividir la nueva columna. Sin embargo, esta no es la intención principal, y solo se haría si es estrictamente necesario.

Este paso contribuye a simplificar el DataFrame y mejorar la eficiencia del análisis posterior.


In [61]:
# Unir las columnas de dirección en una sola

# 1. Selecciona las columnas de dirección ('location.address1', etc.).
# 2. Usa .apply() para aplicar una función a cada fila (axis=1).
# 3. La función lambda toma cada fila 'x' de las columnas de dirección.
# 4. x.dropna() elimina los valores nulos (NaN) de esa fila.
# 5. ', '.join(...) une los valores restantes en una sola cadena, separados por una coma y un espacio.
# 6. El resultado se guarda en una nueva columna llamada 'address'.
df_res['address'] = df_res[['location.address1', 'location.address2', 'location.address3']].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Eliminar las columnas de dirección originales
# 1. df_res.drop() elimina las columnas especificadas.
# 2. columns= [...] indica las columnas a eliminar.
# 3. inplace=True modifica el DataFrame directamente, sin necesidad de reasignarlo.
df_res.drop(columns=['location.address1', 'location.address2', 'location.address3'], inplace=True)

# Muestra el DataFrame modificado
df_res

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,display_phone,distance,coordinates.latitude,coordinates.longitude,location.city,location.zip_code,location.country,location.state,location.display_address,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],...,(312) 492-6262,3401.238676,41.884193,-87.647946,Chicago,60607,US,IL,"['809 W Randolph', 'Chicago, IL 60607']",809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",...,(312) 464-1744,4672.658118,41.890694,-87.624782,Chicago,60611,US,IL,"['444 N Michigan Ave', 'Chicago, IL 60611']",444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",...,(773) 770-3427,2181.370354,41.917275,-87.698577,Chicago,60647,US,IL,"['2833 W Armitage Ave', 'Chicago, IL 60647']",2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],...,(312) 792-3502,5091.190048,41.881689,-87.625006,Chicago,60603,US,IL,"['12 S Michigan Ave', 'Chicago, IL 60603']",12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],...,(312) 796-3316,3939.137304,41.889390,-87.635240,Chicago,60654,US,IL,"['226 W Kinzie', 'Chicago, IL 60654']",226 W Kinzie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,False,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",...,(312) 265-1328,5329.003321,41.890570,-87.616451,Chicago,60611,US,IL,"['465 N McClurg Ct', 'Chicago, IL 60611']",465 N McClurg Ct
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,False,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",...,(773) 799-8585,4257.633138,41.943438,-87.681687,Chicago,60618,US,IL,"['2116 W Roscoe St', 'Chicago, IL 60618']",2116 W Roscoe St
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,False,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",...,(773) 675-0736,10549.492373,41.999260,-87.660308,Chicago,60626,US,IL,"['6431 N Sheridan Rd', 'Chicago, IL 60626']",6431 N Sheridan Rd
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,False,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],...,(872) 274-5862,2857.604229,41.886454,-87.653983,Chicago,60607,US,IL,"['1045 W Fulton St', 'Chicago, IL 60607']",1045 W Fulton St


### Eliminación de Columnas Redundantes: Comparación de 'location.display_address' y 'address'

Continuamos con la eliminación de **columnas redundantes** en el DataFrame. En este caso, notamos que la información contenida en las columnas **'location.display_address'** y **'address'** es muy similar. La diferencia principal es que la información de **'location.display_address'** incluye datos que ya se repiten en otras columnas. Por lo tanto, decidimos quedarnos con una sola de ellas.

Para poder hacer una comparación efectiva entre estas dos columnas y tomar una decisión informada sobre cuál mantener, utilizamos la librería **'fuzzywuzzy'**. Esta librería nos permite realizar una **comparación difusa** entre dos o más variables, lo que es especialmente útil cuando se manejan datos con pequeñas diferencias de formato o errores tipográficos.

**¿Cómo funciona?**
- **Fuzzywuzzy** nos devuelve un **porcentaje de similitud** entre las cadenas comparadas, donde:
  - **0%** indica que no hay coincidencias entre las cadenas.
  - **100%** significa que las cadenas son completamente iguales.
  
Dado que el formato de las direcciones entre **'location.display_address'** y **'address'** puede variar (debido a posibles errores tipográficos o diferencias en la estructura de las direcciones), es importante utilizar esta herramienta para determinar qué tan similares son.

A continuación, se explica con más detalle el procedimiento seguido para:
- **Realizar la comparación** entre las dos columnas.
- **Tomar la decisión** sobre cuál columna mantener basándonos en el porcentaje de similitud.
- **Manejar posibles incongruencias** o diferencias de formato para asegurar que los datos sean consistentes.

Este proceso nos permitió reducir la redundancia de las columnas sin perder información relevante.


### Comparación de Direcciones: Creación de la Columna 'similitud_dirección'

Para proceder con la eliminación de las columnas redundantes, lo primero que hicimos fue crear una nueva columna llamada **'similitud_dirección'**, en la que almacenaremos el **porcentaje de similitud** entre las columnas **'location.display_address'** y **'address'**, fila por fila.

Este paso es esencial para asegurarnos de que las direcciones son efectivamente similares antes de decidir cuál columna eliminar. La comparación se realiza utilizando la librería **'fuzzywuzzy'**, que nos proporciona un valor de similitud en porcentaje, donde:
- **0%** significa que las direcciones no son similares en absoluto.
- **100%** indica que las direcciones son idénticas.

A continuación, se muestran las **primeras 5 filas** de la columna **'similitud_dirección'** para comprobar que el proceso se ha realizado correctamente y que la similitud se calcula adecuadamente entre las direcciones.

Este paso nos permitirá visualizar la calidad de la comparación y determinar si las direcciones son suficientemente similares para proceder con la unificación de las columnas.


In [62]:
# Crear una nueva columna para medir la similitud de las direcciones

# Usa .apply() para procesar cada fila del DataFrame.
df_res['similitud_direccion'] = df_res.apply(
    # La función 'lambda' se ejecuta en cada fila.
    # fuzz.ratio() compara dos cadenas de texto y devuelve un porcentaje de similitud.
    # str(...) se usa para asegurar que los valores sean cadenas, evitando errores.
    lambda row: fuzz.ratio(str(row['location.display_address']), str(row['address'])), 
    
    # axis=1 le indica a la función que debe operar a nivel de filas,
    # es decir, para cada registro del DataFrame.
    axis=1 
)

# Muestra una vista previa del resultado
# Selecciona las columnas de dirección originales y la nueva columna de similitud.
# .head() muestra solo las primeras 5 filas para una revisión rápida.
df_res[['location.display_address', 'address', 'similitud_direccion']].head()

,location.display_address,address,similitud_direccion
0,"['809 W Randolph', 'Chicago, IL 60607']",809 W Randolph,53
1,"['444 N Michigan Ave', 'Chicago, IL 60611']",444 N Michigan Ave,59
2,"['2833 W Armitage Ave', 'Chicago, IL 60647']",2833 W Armitage Ave,60
3,"['12 S Michigan Ave', 'Chicago, IL 60603']",12 S Michigan Ave,58
4,"['226 W Kinzie', 'Chicago, IL 60654']",226 W Kinzie,49


### Revisión de Direcciones con Porcentaje de Similitud Menor al 50%

Una vez creada la columna **'similitud_direccion'** que contiene los valores de similitud entre las columnas **'location.display_address'** y **'address'**, procedemos a revisar aquellas filas en las que el porcentaje de similitud sea **menor al 50%**.

Este umbral se seleccionó porque, debido a las diferencias en el formato de las direcciones, consideramos que un porcentaje de similitud por debajo del 50% indica que las direcciones pueden ser significativamente diferentes o que hay un error en el formato.

Las razones para un porcentaje bajo pueden incluir:
- **Longitud diferente**: En algunos casos, hemos notado que el valor de **'address'** es más corto que el de **'location.display_address'**, lo que genera un valor de similitud más bajo.
  
Para las filas con un porcentaje de similitud menor al 50%, se realizará una **revisión manual** para verificar si la dirección es realmente diferente o si es simplemente un caso de formato.

Este paso es crucial para garantizar que las direcciones que parecen no coincidir realmente lo hagan, y así evitar eliminar o modificar información importante de manera incorrecta.


In [63]:
# Crear una nueva columna para identificar bajas similitudes
# Compara el valor de 'similitud_direccion' en cada fila con el número 50.
# El resultado es un valor booleano (True/False) que se guarda en la nueva columna.
# True: la similitud es menor a 50%.
# False: la similitud es 50% o más.
df_res['similitud_direccion_menor_50'] = df_res['similitud_direccion'] < 50

# Contar la cantidad de True y False en la nueva columna.
# .value_counts() te da un resumen de cuántas filas cumplen con la condición y cuántas no.
df_res['similitud_direccion_menor_50'].value_counts()

similitud_direccion_menor_50
False    196
True       4
Name: count, dtype: int64

In [64]:
df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.city,location.zip_code,location.country,location.state,location.display_address,address,similitud_direccion,similitud_direccion_menor_50
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],...,41.884193,-87.647946,Chicago,60607,US,IL,"['809 W Randolph', 'Chicago, IL 60607']",809 W Randolph,53,False
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",...,41.890694,-87.624782,Chicago,60611,US,IL,"['444 N Michigan Ave', 'Chicago, IL 60611']",444 N Michigan Ave,59,False
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",...,41.917275,-87.698577,Chicago,60647,US,IL,"['2833 W Armitage Ave', 'Chicago, IL 60647']",2833 W Armitage Ave,60,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],...,41.881689,-87.625006,Chicago,60603,US,IL,"['12 S Michigan Ave', 'Chicago, IL 60603']",12 S Michigan Ave,58,False
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],...,41.889390,-87.635240,Chicago,60654,US,IL,"['226 W Kinzie', 'Chicago, IL 60654']",226 W Kinzie,49,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,False,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",...,41.890570,-87.616451,Chicago,60611,US,IL,"['465 N McClurg Ct', 'Chicago, IL 60611']",465 N McClurg Ct,56,False
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,False,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",...,41.943438,-87.681687,Chicago,60618,US,IL,"['2116 W Roscoe St', 'Chicago, IL 60618']",2116 W Roscoe St,56,False
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,False,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",...,41.999260,-87.660308,Chicago,60626,US,IL,"['6431 N Sheridan Rd', 'Chicago, IL 60626']",6431 N Sheridan Rd,59,False
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,False,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],...,41.886454,-87.653983,Chicago,60607,US,IL,"['1045 W Fulton St', 'Chicago, IL 60607']",1045 W Fulton St,56,False


### Eliminación de Columnas Innecesarias

Tras decidir que nos quedaremos con la columna **'address'** (ya que los datos de las otras columnas se repiten o ya se encuentran representados en otras variables), procedemos a **eliminar las columnas que ya no son necesarias**.

Esto incluye:
- Las columnas generadas para la **comparación** de direcciones, como **'similitud_direccion_menor_50'** y **'similitud_direccion'**.
- Otras columnas como **'location.display_address'**, **'location.state'**, **'location.country'**, y **'location.city'**, que no aportan información adicional relevante para el análisis y que ya se encuentran disponibles en otras variables.

Una vez eliminadas estas columnas, **el DataFrame se simplifica**, manteniendo solo la información esencial para un análisis eficiente. Finalmente, mostramos el DataFrame actualizado para verificar que los cambios se hayan aplicado correctamente y que los datos estén listos para su posterior análisis.


In [65]:
# Elimina múltiples columnas del DataFrame en una sola operación.
# El argumento 'columns' recibe una lista de los nombres de las columnas a eliminar.
# 'inplace=True' asegura que los cambios se apliquen directamente al DataFrame 'df_res'.
df_res.drop(
    columns=['similitud_direccion_menor_50', 'similitud_direccion', 'location.display_address', 
             'location.state', 'location.country', 'location.city'], 
    inplace=True
)

# Muestra una vista previa del DataFrame para confirmar que las columnas fueron eliminadas.
df_res

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,1.312493e+10,(312) 492-6262,3401.238676,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,1.312464e+10,(312) 464-1744,4672.658118,41.890694,-87.624782,60611,444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,1.773770e+10,(773) 770-3427,2181.370354,41.917275,-87.698577,60647,2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,1.312792e+10,(312) 792-3502,5091.190048,41.881689,-87.625006,60603,12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,1.312796e+10,(312) 796-3316,3939.137304,41.889390,-87.635240,60654,226 W Kinzie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,False,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,1.312265e+10,(312) 265-1328,5329.003321,41.890570,-87.616451,60611,465 N McClurg Ct
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,False,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,1.773800e+10,(773) 799-8585,4257.633138,41.943438,-87.681687,60618,2116 W Roscoe St
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,False,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,1.773675e+10,(773) 675-0736,10549.492373,41.999260,-87.660308,60626,6431 N Sheridan Rd
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,False,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,1.872275e+10,(872) 274-5862,2857.604229,41.886454,-87.653983,60607,1045 W Fulton St


### Procedemos a hacer lo mismo con las columnas 'display_phone' y 'phone'

In [66]:
# Crea una nueva columna para almacenar el porcentaje de similitud del teléfono.
# Usa el método .apply() para ejecutar una función fila por fila.
df_res['similitud_tel'] = df_res.apply(
    # La función lambda toma cada fila (row).
    # fuzz.ratio() compara dos cadenas y devuelve un valor de 0 a 100.
    # str() se usa para asegurar que los valores sean tratados como texto y evitar errores.
    lambda row: fuzz.ratio(str(row['phone']), str(row['display_phone'])),
    
    # El argumento axis=1 indica que la función debe aplicarse a lo largo de las filas.
    axis=1
)

# Imprime las primeras 5 filas para una revisión rápida.
# Se seleccionan las columnas originales de teléfono y la nueva columna de similitud.
df_res[['phone', 'display_phone', 'similitud_tel']].head()

,phone,display_phone,similitud_tel
0,1.312493e+10,(312) 492-6262,74
1,1.312464e+10,(312) 464-1744,74
2,1.773770e+10,(773) 770-3427,74
3,1.312792e+10,(312) 792-3502,74
4,1.312796e+10,(312) 796-3316,74


### Verificación de Similitudes Menores al 50% en las Columnas de Teléfono

Procedemos a verificar los valores de similitud entre las columnas **'display_phone'** y **'phone'** para identificar aquellos que tengan un porcentaje de similitud menor al **50%**. Sin embargo, al realizar esta revisión, **observamos que todos los valores de similitud son mayores al 70%**, lo que indica que las columnas contienen información muy similar.

Dado que no se encuentran coincidencias por debajo del umbral del 50%, no es necesario realizar ninguna corrección adicional en cuanto a las diferencias de formato en los números de teléfono. Esto nos permite simplificar el análisis y continuar con la limpieza de datos sin complicaciones adicionales.


In [67]:
# Crea una nueva columna booleana ('True' o 'False')
# para cada fila, evalúa si la 'similitud_tel' es menor que 50.
df_res['similitud_tel_menor_50'] = df_res['similitud_tel'] < 50

# Cuenta la cantidad de valores 'True' y 'False' en la nueva columna.
# Esto te da un resumen rápido de cuántos teléfonos tienen un problema de similitud.
df_res['similitud_tel_menor_50'].value_counts()

similitud_tel_menor_50
False    200
Name: count, dtype: int64

### Eliminación de Filas Innecesarias y Conservación de la Columna 'phone'

Con la confirmación de que las columnas **'display_phone'** y **'phone'** coinciden en más del **70%** de sus valores, decidimos proceder con la eliminación de las filas innecesarias. Dado que ambas columnas contienen la misma información, optamos por quedarnos únicamente con la columna **'phone'**, que será suficiente para nuestros análisis futuros.

Este paso nos permite simplificar el DataFrame, eliminando redundancias y asegurándonos de que solo se mantenga la información relevante, lo que mejora la eficiencia y claridad del conjunto de datos.


In [68]:
# Elimina las columnas que ya no se necesitan del DataFrame.
# Se usa 'columns' para especificar la lista de columnas a eliminar.
# 'inplace=True' modifica el DataFrame directamente, sin necesidad de reasignarlo.
df_res.drop(
    columns=['similitud_tel', 'display_phone', 'similitud_tel_menor_50'], 
    inplace=True
)

# Muestra el DataFrame actualizado para verificar que las columnas fueron eliminadas.
df_res

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,distance,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,1.312493e+10,3401.238676,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,1.312464e+10,4672.658118,41.890694,-87.624782,60611,444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,1.773770e+10,2181.370354,41.917275,-87.698577,60647,2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,1.312792e+10,5091.190048,41.881689,-87.625006,60603,12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,1.312796e+10,3939.137304,41.889390,-87.635240,60654,226 W Kinzie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,False,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,1.312265e+10,5329.003321,41.890570,-87.616451,60611,465 N McClurg Ct
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,False,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,1.773800e+10,4257.633138,41.943438,-87.681687,60618,2116 W Roscoe St
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,False,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,1.773675e+10,10549.492373,41.999260,-87.660308,60626,6431 N Sheridan Rd
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,False,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,1.872275e+10,2857.604229,41.886454,-87.653983,60607,1045 W Fulton St


### Cambio de Tipo de Dato en la Columna 'phone'

Además de verificar las similitudes entre las columnas de teléfono, también **cambiamos el tipo de dato de la columna 'phone'**, ya que originalmente se encontraba en formato **'float'**, lo que no es adecuado para almacenar números de teléfono.

### Imputación de Valores Nulos en la Columna 'phone'

Antes de proceder con el cambio de tipo de dato en la columna **'phone'**, **imputamos los valores nulos** con el valor **'0'**. Esto es necesario porque, si la columna contiene valores nulos al intentar cambiar el tipo de dato, se generará un error.

Al imputar los valores nulos con **'0'**, aseguramos que la columna esté completa y podamos realizar la transformación sin inconvenientes. Esta estrategia nos permite evitar errores y facilita el cambio de tipo de dato a **cadena de texto** (string), ya que los números de teléfono deben tratarse de esta forma para evitar interpretaciones incorrectas.

Una vez imputados los valores nulos, procedemos con el cambio de tipo de dato de la columna.


In [69]:
# Imputa los valores nulos en la columna 'phone'.
# .fillna('0') reemplaza todos los valores NaN con la cadena de texto '0'.
# Esto evita problemas si luego intentas convertir la columna a un tipo numérico,
# ya que la mayoría de los tipos de datos numéricos no pueden manejar NaN.
df_res['phone'] = df_res['phone'].fillna('0')

# Muestra un resumen del DataFrame.
# Esto te permite verificar que la columna 'phone' ya no tiene valores nulos
# y te muestra su tipo de dato actual (probablemente 'object' si tenía texto).
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   name                   200 non-null    object 
 3   image_url              200 non-null    object 
 4   is_closed              200 non-null    bool   
 5   url                    200 non-null    object 
 6   review_count           200 non-null    int64  
 7   categories             200 non-null    object 
 8   rating                 200 non-null    float64
 9   transactions           200 non-null    object 
 10  price                  134 non-null    object 
 11  phone                  200 non-null    object 
 12  distance               200 non-null    float64
 13  coordinates.latitude   200 non-null    float64
 14  coordinates.longitude  200 non-null    float64
 15  locati

### Conversión de Datos a Tipo 'int'

Una vez que hemos imputado los valores nulos y nos aseguramos de que la columna **'phone'** esté completa, procedemos a convertir los datos a tipo **'int'**. Esta conversión se realiza con el objetivo de eliminar los **puntos decimales** presentes en la columna, ya que los números de teléfono no deberían tener decimales.

Al convertir los datos a **'int'**, nos aseguramos de que la columna **'phone'** esté correctamente formateada como números enteros, lo que también facilita la manipulación de los datos y garantiza que no h


In [70]:
# Convierte el tipo de dato de la columna 'phone' a entero (int).
# Esta operación requiere que todos los valores sean numéricos (ej. no tener texto ni valores nulos).
df_res['phone'] = df_res['phone'].astype(int)

# Muestra un resumen del DataFrame para verificar el cambio.
# Ahora la columna 'phone' debería mostrar un 'Dtype' de 'int64'.
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   name                   200 non-null    object 
 3   image_url              200 non-null    object 
 4   is_closed              200 non-null    bool   
 5   url                    200 non-null    object 
 6   review_count           200 non-null    int64  
 7   categories             200 non-null    object 
 8   rating                 200 non-null    float64
 9   transactions           200 non-null    object 
 10  price                  134 non-null    object 
 11  phone                  200 non-null    int64  
 12  distance               200 non-null    float64
 13  coordinates.latitude   200 non-null    float64
 14  coordinates.longitude  200 non-null    float64
 15  locati

### Cambio Final de Tipo de Dato a 'str'

Después de haber imputado los valores nulos y eliminado los puntos decimales al convertir los datos a **'int'**, finalmente **cambiamos el tipo de dato a 'str'**. Esta transformación es necesaria porque **no se realizará ninguna operación matemática** con los números de teléfono. Al tratarlos como cadenas de texto, podemos mantener su formato original, incluyendo los posibles caracteres especiales como guiones o paréntesis.

Este cambio asegura que los datos de la columna **'phone'** se gestionen correctamente como texto y no como valores numéricos, lo que evita errores y facilita su manipulación y visualización en el futuro.


In [71]:
# Convierte el tipo de dato de la columna 'phone' a string (str).
# Esto es útil para prevenir operaciones matemáticas accidentales en los números de teléfono.
df_res['phone'] = df_res['phone'].astype(str)

# Muestra un resumen del DataFrame para verificar el cambio.
# Ahora la columna 'phone' debería mostrar un 'Dtype' de 'object'.
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   name                   200 non-null    object 
 3   image_url              200 non-null    object 
 4   is_closed              200 non-null    bool   
 5   url                    200 non-null    object 
 6   review_count           200 non-null    int64  
 7   categories             200 non-null    object 
 8   rating                 200 non-null    float64
 9   transactions           200 non-null    object 
 10  price                  134 non-null    object 
 11  phone                  200 non-null    object 
 12  distance               200 non-null    float64
 13  coordinates.latitude   200 non-null    float64
 14  coordinates.longitude  200 non-null    float64
 15  locati

### Eliminación de la Columna 'distance'

Continuamos con el proceso de limpieza del DataFrame y, al revisar las columnas, notamos que la columna **'distance'** parecía no aportar información relevante para el análisis. Decidimos investigar su contenido para entender mejor a qué hacía referencia.

Tras realizar la investigación, descubrimos que **'distance'** representa la distancia entre el lugar donde se generó la **API Key** y la ubicación del restaurante. Dado que esta información no es relevante para el análisis que estamos llevando a cabo, y no nos aporta valor adicional en nuestro caso, se tomó la decisión de **eliminarla**.

Este paso contribuye a simplificar el DataFrame, eliminando columnas que no son útiles para los objetivos


In [72]:
# Elimina la columna 'distance' del DataFrame.
# 'inplace=True' modifica el DataFrame directamente sin crear una nueva copia.
df_res.drop(columns=['distance'], inplace=True)

# Muestra el DataFrame actualizado para verificar que la columna fue eliminada.
df_res

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,False,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,False,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,False,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,False,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,False,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,False,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,False,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St


### Eliminación de la Columna 'is_closed'

Durante la primera fase de exploración, notamos que en la columna **'is_closed'** solo había un único valor que se repetía en todos los registros. Para asegurarnos de que esta información no aportaba valor al análisis, procedimos a corroborar su contenido.

Al verificar los datos, confirmamos que **todos los restaurantes** tenían la misma información en esta columna, indicando que el valor era consistente en todos los casos. Dado que no agrega ninguna variabilidad ni utilidad a nuestro análisis, decidimos **eliminar la columna**.

Este paso es parte del proceso de simplificación del DataFrame, eliminando variables que no aportan información relevante y mejorando la eficiencia del análisis.



In [73]:
# contar los falses de is_closed

df_res['is_closed'].value_counts() # Contar los valores únicos en la columna 'is_closed' para ver cuántos son True y cuántos son False

is_closed
False    200
Name: count, dtype: int64

In [74]:
# Se elimina la columna is_closed

df_res.drop(columns=['is_closed'], inplace=True) 

df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St


### Comparación de las Columnas 'alias' y 'name'

Al igual que en instancias anteriores, notamos que las columnas **'alias'** y **'name'** parecen tener información muy similar, ya que ambas hacen referencia al **nombre del restaurante**. Para verificar si realmente contienen los mismos datos o si hay alguna diferencia, decidimos aplicar el mismo enfoque utilizado anteriormente con la librería **'fuzzywuzzy'** para realizar una comparación difusa entre ambas columnas.

Este proceso nos permite calcular un **porcentaje de similitud** entre los valores de **'alias'** y **'name'** y determinar si podemos consolidar estas columnas en una sola, eliminando redundancias en el DataFrame.

Al igual que con las comparaciones anteriores, utilizamos **fuzzywuzzy** para garantizar que los datos sean consistentes y evitar errores de formato o diferencias menores que puedan existir entre los valores de las columnas.


In [75]:
# Crea una nueva columna para almacenar el porcentaje de similitud.
# Usa el método .apply() para ejecutar la función fila por fila.
df_res['similitud_nombre'] = df_res.apply(
    # La función lambda toma cada fila.
    # fuzz.ratio() compara las cadenas de texto 'alias' y 'name' y devuelve un valor de 0 a 100.
    # str() se usa para asegurar que los valores sean texto y evitar errores.
    lambda row: fuzz.ratio(str(row['alias']), str(row['name'])),
    
    # axis=1 le indica a pandas que la función debe aplicarse a lo largo de las filas.
    axis=1
)

# Imprime las primeras 5 filas para una revisión rápida.
# Se seleccionan las columnas 'alias', 'name' y la nueva columna de similitud.
df_res[['alias', 'name', 'similitud_nombre']].head()

,alias,name,similitud_nombre
0,girl-and-the-goat-chicago,Girl & The Goat,40
1,the-purple-pig-chicago,The Purple Pig,50
2,gretel-chicago,Gretel,50
3,cindys-rooftop-chicago,Cindy's Rooftop,59
4,ciccio-mio-chicago-2,Ciccio Mio,47


### Diferencias de Formato y Similitudes Menores al 50%

Al aplicar el método de comparación entre las columnas **'alias'** y **'name'**, nos damos cuenta de que, debido a las diferencias en el formato en el que están escritos los nombres de los restaurantes, una mayor cantidad de registros presenta **porcentajes de similitud menores al 50%**.

Estas diferencias de formato pueden incluir variaciones en la puntuación, el uso de mayúsculas o minúsculas, o la presencia de caracteres especiales que afectan la similitud entre los valores. Por lo tanto, los resultados de comparación muestran una mayor cantidad de valores con similitud baja, lo que indica que, aunque los nombres representan el mismo restaurante, las variaciones de formato afectan la puntuación de similitud.

Este hallazgo sugiere que la comparación directa entre estas columnas podría no ser suficiente para consolidarlas sin una limpieza adicional de formato. Será necesario revisar y estandarizar los valores antes de tomar decisiones finales sobre la eliminación o consolidación de estas columnas.


In [76]:
# Crea una nueva columna booleana que es 'True' si la similitud es menor al 50%.
df_res['similitud_nombre_menor_50'] = df_res['similitud_nombre'] < 50

# Cuenta y muestra la cantidad de valores 'True' y 'False' en la nueva columna.
# Esto te da un resumen rápido de cuántas filas cumplen con la condición.
df_res['similitud_nombre_menor_50'].value_counts()

similitud_nombre_menor_50
False    143
True      57
Name: count, dtype: int64

In [77]:
df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,name,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,similitud_nombre,similitud_nombre_menor_50
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,40,True
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,50,False
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,Gretel,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,50,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,Cindy's Rooftop,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,59,False
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,Ciccio Mio,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,47,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,Robert's Pizza and Dough,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,62,False
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,Dhanteraz Indian Fusion,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,67,False
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,Cafe Nova,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,46,True
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,Il Carciofo,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,50,False


### Decisión de Eliminar la Columna 'name' y Conservar 'alias'

Debido a que una gran cantidad de los valores de similitud entre las columnas **'alias'** y **'name'** son menores al 50%, decidimos realizar una **inspección visual** de los datos para determinar si es necesario eliminar o modificar alguna fila en particular. Al revisar los resultados, llegamos a la conclusión de que ambas columnas contienen la misma información, pero que **'alias'** tiene más detalles y es más completa.

Con base en esta observación, decidimos que no es necesario mantener ambas columnas, ya que **'alias'** proporciona la misma información de manera más detallada. Por lo tanto, **eliminamos la columna 'name'** y decidimos **quedarnos con la columna 'alias'**, que será suficiente para nuestros análisis posteriores.

Esta acción simplifica el DataFrame y elimina redundancias, manteniendo solo la columna más relevante.


In [78]:
# Elimina las columnas que ya no se necesitan del DataFrame.
# Se usa 'columns' para especificar la lista de columnas a eliminar.
# 'inplace=True' modifica el DataFrame directamente, sin necesidad de reasignarlo.
df_res.drop(
    columns=['name', 'similitud_nombre', 'similitud_nombre_menor_50'],
    inplace=True
)

# Muestra el DataFrame actualizado para verificar que las columnas fueron eliminadas.
df_res

,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St


### Transformación de la Columna 'categories'

Al analizar la columna **'categories'**, nos dimos cuenta de que el tipo de dato era un **objeto**, pero en realidad se trataba de una cadena de texto (**'str'**). Esto nos dificultaba el trabajo, ya que necesitamos manipular las categorías de manera individual. 

Para solucionar esto, utilizamos la librería **'ast'**, que nos permite convertir estos datos, originalmente representados como cadenas en el DataFrame de Pandas, a estructuras nativas de Python. Esta conversión nos permite trabajar con los datos de manera más eficiente y facilita futuras transformaciones o análisis relacionados con las categorías de los restaurantes.



### Conversión del Tipo de Dato de 'str' a 'lista'

Comenzamos cambiando el tipo de dato de **'str'** a **'lista'**, ya que originalmente los valores en la columna **'categories'** eran cadenas de texto con formato de lista de diccionarios. Al aplicar esta conversión, ahora los datos se encuentran en el formato adecuado, permitiéndonos trabajar con ellos como **listas** de manera eficiente.



In [79]:
# Convierte la columna 'categories' (que es una cadena de texto) en una lista de Python.
# 1. df_res["categories"] = ... : Asigna el resultado de la operación de vuelta a la misma columna.
# 2. .apply(ast.literal_eval): Itera sobre cada valor de la columna y aplica la función ast.literal_eval.
#    Esta función es segura y se usa para evaluar cadenas de texto que contienen
#    estructuras de datos de Python (como listas o diccionarios).
df_res["categories"] = df_res["categories"].apply(ast.literal_eval)

In [80]:
df_res.head() # Mostramos el DataFrame actualizado para verificar los cambios


,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie


### Extracción de las Claves de los Diccionarios en la Columna 'categories'

A continuación, procedemos a extraer las **claves de los diccionarios** que se encuentran dentro de la lista en cada fila de la columna **'categories'**. Este paso nos permite obtener solo las **categorías** presentes en cada restaurante, eliminando el resto de la información contenida en los diccionarios.

De esta manera, transformamos los datos para que cada categoría sea fácilmente accesible y utilizable en análisis posteriores.


In [81]:
# Extrae los valores de los diccionarios dentro de la lista de categorías
# y los guarda en una nueva columna.

# df_res["category_keys"] = ... : Asigna el resultado a una nueva columna.
# .apply(lambda lst: ...): Aplica una función a cada fila de la columna 'categories'.
#
# La función lambda:
#   - Toma 'lst', que es una lista de diccionarios (ej. [{'alias': '...', 'title': '...'}, ...]).
#   - [list(d.values()) for d in lst]: Itera sobre cada diccionario ('d') en la lista ('lst').
#   - d.values(): Obtiene los valores de cada diccionario (ej. 'mexican', 'Mexican').
#   - list(...): Convierte esos valores en una lista.
df_res["category_keys"] = df_res["categories"].apply(lambda lst: [list(d.values()) for d in lst])

In [82]:
df_res.head() # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,category_keys
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[[newamerican, New American], [bars, Bars], [b..."
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[[tapasmallplates, Tapas/Small Plates], [medit..."
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[[cocktailbars, Cocktail Bars], [newamerican, ..."
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[[newamerican, New American], [seafood, Seafoo..."
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,"[[italian, Italian]]"


### Selección de los Primeros Valores de Cada Sublista

Una vez que hemos extraído las claves de los diccionarios en la columna **'categories'**, procedemos a tomar **solo los primeros valores de cada sublista**. Esto se debe a que, tras la extracción, los valores en las sublistas hacen referencia a la misma categoría. Al seleccionar solo el primer valor, garantizamos que mantenemos la información en un formato más limpio y accesible.

Además, al hacerlo, los **alias** se presentan de manera más manejable y organizada, facilitando su uso en análisis posteriores y asegurando que la información esté correctamente estructurada.


In [83]:
# Crea una nueva columna para almacenar los alias de las categorías.
# .apply(lambda lst: ...): Aplica una función a cada fila de la columna 'category_keys'.
#
# La función lambda:
#   - Toma 'lst', que es una lista de sublistas (ej. [['mexican', 'Mexican'], ...]).
#   - [sub[0] for sub in lst]: Itera sobre cada sublista ('sub') en la lista.
#   - sub[0]: Extrae el primer elemento de cada sublista (el alias).
df_res["aliases"] = df_res["category_keys"].apply(lambda lst: [sub[0] for sub in lst])

In [84]:
df_res # Mostramos el DataFrame actualizado para verificar los cambios

,id,alias,image_url,url,review_count,categories,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,category_keys,aliases
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[[newamerican, New American], [bars, Bars], [b...","[newamerican, bars, bakeries]"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[[tapasmallplates, Tapas/Small Plates], [medit...","[tapasmallplates, mediterranean, newamerican]"
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[[cocktailbars, Cocktail Bars], [newamerican, ...","[cocktailbars, newamerican, speakeasies]"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,"[{'alias': 'newamerican', 'title': 'New Americ...",4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[[newamerican, New American], [seafood, Seafoo...","[newamerican, seafood, breakfast_brunch]"
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,"[[italian, Italian]]",[italian]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.4,"['pickup', 'delivery']",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,"[[pizza, Pizza]]",[pizza]
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,"[{'alias': 'indpak', 'title': 'Indian'}]",4.7,"['pickup', 'delivery']",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,"[[indpak, Indian]]",[indpak]
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,"[{'alias': 'srilankan', 'title': 'Sri Lankan'}...",4.6,"['pickup', 'delivery']",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[[srilankan, Sri Lankan], [indpak, Indian]]","[srilankan, indpak]"
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[[italian, Italian], [desserts, Desserts], [wi...","[italian, desserts, wine_bars]"


### Eliminación de Columnas Innecesarias

Una vez que hemos transformado las categorías a un formato más manejable, **procedemos a eliminar las columnas que ya no consideramos necesarias**. Estas columnas incluyen aquellas que contenían información redundante o que ya hemos consolidado en un formato más accesible para su análisis.

Este paso nos permite simplificar aún más el DataFrame, eliminando datos innecesarios y asegurando que solo se mantenga la información relevante para el análisis posterior.


In [85]:
# Elimina las columnas que ya no se necesitan del DataFrame.
# Se usa 'columns' para especificar la lista de columnas a eliminar.
# 'inplace=True' modifica el DataFrame directamente, sin necesidad de reasignarlo.
df_res.drop(
    columns=['category_keys', 'categories'], 
    inplace=True
)

# Muestra el DataFrame actualizado para verificar que las columnas fueron eliminadas.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,['delivery'],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"['pickup', 'delivery']",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]"
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"['pickup', 'delivery']",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,['delivery'],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]"
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,['delivery'],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"['pickup', 'delivery']",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza]
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"['pickup', 'delivery']",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak]
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"['pickup', 'delivery']",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]"
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]"


### Cambio de Tipo de Dato de 'transactions'

De manera similar a lo que hicimos con la columna **'categories'**, procedemos a cambiar el tipo de dato de **'transactions'**. Inicialmente, los valores en esta columna están en formato **'str'**, pero representan una lista de cadenas de texto. Para poder trabajar con estos datos de manera más efectiva, transformamos este **'str'** en una **lista real**, lo que facilita el manejo y análisis de las transacciones.

Este cambio nos permite acceder a los valores individuales de cada lista de transacciones, mejorando la estructura de los datos para futuros análisis.


In [86]:
df_res["transactions"] = df_res["transactions"].apply(ast.literal_eval) # Cambiar el tipo de dato de 'transactions' de 'str' a 'lista'


In [87]:
df_res.describe(include='all') # Revisamos el cambio de tipo de dato en la columna 'transactions'

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases
count,200,200,200,200,200.000000,200.000000,200,134,200,200.000000,200.000000,200.000000,200,200
unique,200,200,200,200,NaN,NaN,12,4,190,NaN,NaN,NaN,197,173
top,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,NaN,NaN,"[pickup, delivery]",$$,0,NaN,NaN,NaN,59 W Hubbard St,[italian]
freq,1,1,1,1,NaN,NaN,84,75,10,NaN,NaN,NaN,2,9
mean,NaN,NaN,NaN,NaN,876.090000,4.401500,NaN,NaN,NaN,41.905545,-87.657499,60624.730000,NaN,NaN
std,NaN,NaN,NaN,NaN,1620.164432,0.231768,NaN,NaN,NaN,0.026011,0.028987,29.663051,NaN,NaN
min,NaN,NaN,NaN,NaN,4.000000,3.800000,NaN,NaN,NaN,41.848943,-87.802095,60302.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,101.750000,4.300000,NaN,NaN,NaN,41.888774,-87.677499,60611.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,307.000000,4.400000,NaN,NaN,NaN,41.900134,-87.650453,60622.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,838.250000,4.500000,NaN,NaN,NaN,41.920080,-87.632185,60647.000000,NaN,NaN


### Manejo de Valor Nulo en la Columna 'image'

Continuando con el tratamiento de valores que podrían causar problemas, notamos que existe un **valor nulo** en la columna **'image'**. Dado que estamos trabajando para una empresa cuyo enfoque principal es el **marketing digital**, decidimos eliminar este único valor nulo. Las imágenes son un componente fundamental en este campo, y la ausencia de una imagen en un registro puede afectar la calidad del análisis.

Además, al ser solo un valor nulo en una columna que contiene datos clave para nuestra empresa, **no representa una pérdida significativa de datos**. Por lo tanto, se tomó la decisión de eliminar esta fila para mantener la integridad y coherencia de los datos.


In [88]:
# Elimina las filas donde el valor en la columna 'image_url' es nulo.
# 'subset' le dice a pandas en qué columna buscar los valores nulos.
df_res = df_res.dropna(subset=['image_url'])

# Muestra un resumen del DataFrame para verificar el resultado.
# La cuenta de valores no nulos en 'image_url' debería ser igual al total de filas.
# Como se indicó, el resultado final es un DataFrame con 199 filas.
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   image_url              200 non-null    object 
 3   url                    200 non-null    object 
 4   review_count           200 non-null    int64  
 5   rating                 200 non-null    float64
 6   transactions           200 non-null    object 
 7   price                  134 non-null    object 
 8   phone                  200 non-null    object 
 9   coordinates.latitude   200 non-null    float64
 10  coordinates.longitude  200 non-null    float64
 11  location.zip_code      200 non-null    int64  
 12  address                200 non-null    object 
 13  aliases                200 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.0+ KB


### Tratamiento de Valores Nulos en la Columna 'price'

Para finalizar con la limpieza de este DataFrame, nos enfocamos en la columna **'price'**, donde observamos que existen **70 valores nulos**. Dado que el DataFrame es relativamente pequeño, eliminar estos valores o simplemente ignorarlos podría resultar en la pérdida de **cerca del 35% de los datos de esta columna**, lo que representaría un riesgo para la calidad de los análisis futuros.

Es por ello que decidimos **imputar los valores nulos** de manera adecuada para no perder información crucial. A continuación, se detallan los pasos seguidos para realizar este tratamiento y garantizar que los datos de la columna **'price'** sean completos y útiles para los análisis posteriores.


### Identificación de Valores Nulos y Frecuencia en la Columna 'price'

El primer paso para tratar los valores nulos en la columna **'price'** fue **identificar los valores nulos** presentes en la columna. Esto nos permitió conocer cuántos registros faltaban información en esta categoría.

Además, generamos la **tabla de frecuencia** de los valores presentes en la columna **'price'**. Es importante destacar que para esta tabla de frecuencia, **no se toman en cuenta los valores nulos**, ya que estamos interesados solo en los datos disponibles. Esto nos permite entender mejor la distribución de los precios y cómo se distribuyen los diferentes rangos dentro de la columna.

Este análisis preliminar nos proporciona una visión clara de los datos existentes, lo cual es crucial antes de proceder con la imputación de los valores faltantes.


In [89]:
# Muestra todos los valores únicos en la columna 'price'.
# Esto te ayuda a entender qué tipo de datos contiene (por ejemplo, '$', '$$', etc.).
df_res['price'].unique()

array(['$$$', '$$', nan, '$$$$', '$'], dtype=object)

In [90]:
# Cuenta la frecuencia de cada valor único en la columna 'price'.
conteo_valores = df_res['price'].value_counts()

# Muestra el resultado de la cuenta.
# Por defecto, los valores se ordenan de mayor a menor frecuencia.
conteo_valores

price
$$      75
$$$     47
$$$$    11
$        1
Name: count, dtype: int64

In [91]:
# Importa la librería 'itertools', que proporciona funciones para trabajar con iteradores.
import itertools

# Aplana la lista de listas en la columna 'aliases' a una sola lista.
# itertools.chain.from_iterable() es una forma muy eficiente de concatenar
# todos los elementos de una lista de listas en una sola lista plana.
todos_aliases = list(itertools.chain.from_iterable(df_res['aliases']))

# Convierte la lista plana de aliases a un conjunto (set).
# Los conjuntos solo pueden contener valores únicos, lo que elimina los duplicados.
valores_unicos = set(todos_aliases)

# Imprime la cantidad de aliases únicos encontrados.
print(f"Número de aliases únicos: {len(valores_unicos)}")

# Muestra los aliases únicos.
print(valores_unicos)

Número de aliases únicos: 101
{'burgers', 'singaporean', 'fishnchips', 'falafel', 'british', 'coffee', 'delis', 'chicken_wings', 'supperclubs', 'pastashops', 'vegetarian', 'mideastern', 'noodles', 'seafood', 'japacurry', 'indpak', 'bbq', 'piadina', 'sandwiches', 'pancakes', 'lebanese', 'latin', 'dimsum', 'salad', 'argentine', 'filipino', 'chinese', 'pubs', 'german', 'ramen', 'japanese', 'poke', 'whiskeybars', 'chickenshop', 'waffles', 'breakfast_brunch', 'kebab', 'desserts', 'halal', 'soup', 'hotpot', 'beer_and_wine', 'brewpubs', 'tapasmallplates', 'sardinian', 'speakeasies', 'peruvian', 'cafes', 'vietnamese', 'spanish', 'gastropubs', 'italian', 'newamerican', 'comfortfood', 'restaurants', 'hainan', 'srilankan', 'beerbar', 'bars', 'bakeries', 'asianfusion', 'karaoke', 'pizza', 'sushi', 'lounges', 'wine_bars', 'venues', 'cajun', 'brasseries', 'cambodian', 'intlgrocery', 'izakaya', 'ukrainian', 'mediterranean', 'mexican', 'thai', 'african', 'malaysian', 'turkish', 'greek', 'french', 'ice

### Imputación de Valores Nulos en la Columna 'price'

Con los datos obtenidos del análisis preliminar y evaluando la posible correlación con otras variables, decidimos imputar los valores nulos en la columna **'price'** utilizando la **media de los valores por alias** (en este caso, las categorías de los restaurantes). 

Esta decisión se tomó considerando que las otras variables no presentan una relación suficientemente fuerte con el precio, lo que podría generar sesgos o falacias en los resultados. Por ejemplo, aunque la **ubicación** de un restaurante podría influir en su precio, no está directamente relacionada con su valor en todos los casos. Lo mismo ocurre con el **rating** o las **transacciones**, que no tienen una conexión directa y contundente con el precio.

Sin embargo, **el tipo de comida** es un indicador clave en este análisis, ya que las categorías de los restaurantes afectan directamente los rangos de precios. Por lo tanto, decidimos imputar los valores faltantes de **'price'** basándonos en la media de los precios dentro de la misma categoría de restaurante, asegurando una imputación más coherente y relevante para los análisis posteriores.


### Extracción de Categorías y Transformación en Cadena

Para comenzar con la imputación de los valores nulos en la columna **'price'**, primero procedimos a **extraer las categorías de los restaurantes** de las listas en las que estaban almacenadas. Esta extracción nos permitió obtener una **única cadena** que contiene todas las categorías separadas por comas (`,`).

De esta forma, transformamos los datos de una estructura de lista a una representación más simple y manejable, lo que facilita el análisis y la imputación de los valores faltantes. Al tener las categorías en formato de cadena, podemos agrupar fácilmente los restaurantes por tipo de comida y calcular la media de los precios de manera más eficiente.


In [92]:
# Crea una nueva columna llamada 'aliases_2'.
# Aplica una función lambda a cada fila de la columna 'aliases'.
# La función lambda toma la lista de aliases (x) y los une en una sola cadena.
# El método .join() une los elementos de una lista con un separador, en este caso ', '.
# map(str, x) se asegura de que todos los elementos de la lista sean tratados como cadenas antes de unirlos.
df_res['aliases_2'] = df_res['aliases'].apply(lambda x: ', '.join(map(str, x)))

# Muestra el DataFrame actualizado.
# Esto te permite ver inmediatamente la nueva columna 'aliases_2' con los datos transformados.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries"
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"[pickup, delivery]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican"
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"[pickup, delivery]",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]","cocktailbars, newamerican, speakeasies"
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch"
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,[delivery],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian],italian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"[pickup, delivery]",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza],pizza
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"[pickup, delivery]",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak],indpak
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"[pickup, delivery]",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]","srilankan, indpak"
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]","italian, desserts, wine_bars"


### Conversión del Precio a Valor Numérico

Una vez que extrajimos las categorías y las transformamos en una cadena, procedimos a **convertir el precio a un valor numérico**. Esta conversión es esencial para facilitar su manipulación y permitirnos realizar diversas operaciones con los datos de manera eficiente.

Al tener el precio en formato numérico, podemos calcular la **media de los precios** por categoría, lo que será fundamental para imputar los valores nulos de la columna **'price'**. Esta transformación nos permite trabajar con los datos de manera más fluida y aplicar técnicas de análisis más complejas sin limitaciones de formato.


In [93]:
# Convierte los símbolos de precio en un valor numérico.
# 1. .map(...) aplica una función a cada elemento de la columna 'price'.
# 2. lambda x: len(str(x)) : para cada valor 'x', calcula la longitud de la cadena.
#    Por ejemplo, '$' se convierte a 1, '$$' a 2, etc.
# 3. if pd.notna(x) else np.nan : Esta es una condición crucial que verifica si el valor no es nulo.
#    Si es nulo, lo mantiene como nulo (np.nan), lo que evita errores.
df_res['price_num'] = df_res['price'].map(lambda x: len(str(x)) if pd.notna(x) else np.nan)

# Muestra el DataFrame actualizado para verificar que la nueva columna 'price_num' se creó correctamente.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.0
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"[pickup, delivery]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.0
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"[pickup, delivery]",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]","cocktailbars, newamerican, speakeasies",2.0
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.0
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,[delivery],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian],italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"[pickup, delivery]",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza],pizza,2.0
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"[pickup, delivery]",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak],indpak,NaN
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"[pickup, delivery]",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]","srilankan, indpak",2.0
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]","italian, desserts, wine_bars",NaN


## Transformación del DataFrame: aplanamiento de `aliases` y `price_num`

Cuando **aplanamos** las columnas `aliases` y `price_num`, lo que hacemos es:

- Convertir cada **alias** (categoría de restaurante) en una fila independiente.  
- Asociar a cada alias su valor numérico de **precio** correspondiente.  

De esta manera, el DataFrame queda más claro y estructurado, lo que nos permite:

- Analizar con mayor detalle la relación entre las **categorías gastronómicas** y sus **rangos de precios**.  
- Explorar los datos de forma más **flexible y precisa**.  



In [94]:
# Crea un nuevo DataFrame seleccionando las columnas 'aliases' y 'price_num'.
# Explode() toma cada lista en la columna 'aliases' y crea una nueva fila para cada elemento de esa lista.
# reset_index(drop=True) reinicia el índice del nuevo DataFrame para que sea consecutivo.
df_alias_price = df_res[['aliases', 'price_num']].explode('aliases').reset_index(drop=True)

# Muestra el DataFrame resultante para confirmar que la transformación se realizó correctamente.
display(df_alias_price)

,aliases,price_num
0,newamerican,3.0
1,bars,3.0
2,bakeries,3.0
3,tapasmallplates,3.0
4,mediterranean,3.0
...,...,...
462,desserts,NaN
463,wine_bars,NaN
464,ukrainian,2.0
465,breakfast_brunch,2.0


## Cálculo del precio promedio por categoría

Una vez obtenida la nueva tabla aplanada, se calcula el promedio de `price_num` para cada alias (categoría de restaurante).  

De esta manera:  
- Se identifica el rango de precios característico de cada tipo de restaurante.  
- Esta información permite imputar valores faltantes de manera más coherente y precisa.  



In [95]:
# Agrupa el DataFrame por la columna 'aliases'.
# Calcula el promedio (mean()) de la columna 'price_num' para cada grupo.
# reset_index() convierte el resultado de nuevo en un DataFrame.
media_por_alias = df_alias_price.groupby('aliases')['price_num'].mean().reset_index()

# Muestra el DataFrame resultante, que contiene el precio promedio por alias.
media_por_alias

,aliases,price_num
0,african,1.0
1,argentine,2.0
2,asianfusion,2.1
3,australian,NaN
4,bakeries,2.5
...,...,...
96,venues,2.2
97,vietnamese,2.0
98,waffles,2.0
99,whiskeybars,2.0


## Creación de un diccionario de alias y precios promedio

A partir de los cálculos realizados, se construye un diccionario que asocia cada **alias** (categoría de restaurante) con su **precio promedio**.  

Este diccionario permite:  
- Utilizar la información de forma más eficiente.  
- Acceder rápidamente al rango de precios característico de cada categoría.  


In [96]:
# Crea un diccionario para mapear cada alias de categoría a su precio promedio.
# 1. .set_index('aliases'): Establece la columna 'aliases' como el índice del DataFrame.
# 2. ['price_num']: Selecciona la columna 'price_num'.
# 3. .to_dict(): Convierte la Serie resultante en un diccionario de Python.
media_por_alias_dict = media_por_alias.set_index('aliases')['price_num'].to_dict()

## Imputación de valores faltantes en `price_num`

Con el diccionario de precios promedio, se define una función para imputar los valores faltantes de `price_num` en el DataFrame original.  

La lógica aplicada es la siguiente:  
- Si el valor de `price_num` es nulo, se asigna la **media de los precios** correspondiente a las categorías (aliases) del restaurante.  
- De esta manera, la imputación resulta más **informada** y **específica** que utilizar un valor genérico.  


In [97]:
# Define una función para imputar valores nulos en la columna 'price_num'.
def imputar_media_aliases(row):
    
    # Condición principal: solo actúa si el valor de 'price_num' es nulo.
    if pd.isna(row['price_num']):
        
        # Obtiene la lista de aliases del restaurante en la fila actual.
        aliases = row['aliases']
        
        # Crea una lista de los precios promedio de los aliases del restaurante.
        # Solo considera aliases que están en el diccionario y que no tienen un valor nulo.
        medias = [media_por_alias_dict[a] for a in aliases if a in media_por_alias_dict and not pd.isna(media_por_alias_dict[a])]
        
        # Si la lista de medias no está vacía (es decir, si encontró un alias válido),
        # calcula y retorna la media de esas medias.
        if medias:
            return np.mean(medias)
    
    # Si el valor original de 'price_num' no era nulo, simplemente lo retorna.
    return row['price_num']

## Aplicación de la función de imputación

Una vez definida la función para imputar los valores faltantes de `price_num`, se procede a aplicarla sobre el DataFrame original.  

Con ello se reemplazan los valores nulos por el **promedio de precios** asociado a las categorías (aliases) de cada restaurante, logrando un conjunto de datos más completo y consistente.  


In [98]:
# Aplica la función 'imputar_media_aliases' a cada fila del DataFrame.
# axis=1 asegura que la función se ejecute fila por fila.
# El resultado de la función se asigna de nuevo a la columna 'price_num'.
df_res['price_num'] = df_res.apply(imputar_media_aliases, axis=1)

# Muestra el DataFrame actualizado para verificar que los valores nulos fueron imputados.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.000000
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"[pickup, delivery]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.000000
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"[pickup, delivery]",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]","cocktailbars, newamerican, speakeasies",2.000000
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.000000
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,[delivery],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian],italian,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"[pickup, delivery]",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza],pizza,2.000000
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"[pickup, delivery]",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak],indpak,2.000000
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"[pickup, delivery]",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]","srilankan, indpak",2.000000
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]","italian, desserts, wine_bars",2.567669


## Verificación de la imputación y análisis de valores restantes

Tras aplicar la función de imputación, se comprueba que los valores nulos en `price_num` se redujeron significativamente.  
No obstante, aún persisten algunos valores faltantes.  

Esto no se debe a un error, sino a la ausencia de información para ciertas categorías, lo que impide asignarles un precio promedio mediante la función definida.  

Frente a esta situación, se procede a:  
- Analizar cuántos valores nulos permanecen en la columna `price_num`.  
- Evaluar si resulta conveniente imputarlos utilizando la **media global** de la columna.  


In [99]:
# cuantos valores nulos quedan en price_num

df_res['price_num'].isna().sum()

np.int64(3)

Debido a que el porcentaje de nulos es bastante bajo (3%) el imputarlos con la media global no representa un gran sesgo

In [100]:
# Rellena los valores nulos ('NaN') en la columna 'price_num'.
# Se usa .mean() para obtener la media global de la columna.
# 'inplace=True' modifica el DataFrame directamente.
df_res['price_num'] = df_res['price_num'].fillna(df_res['price_num'].mean())

# Muestra el DataFrame actualizado para verificar que no queden valores nulos.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.000000
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"[pickup, delivery]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.000000
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"[pickup, delivery]",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]","cocktailbars, newamerican, speakeasies",2.000000
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.000000
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,[delivery],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian],italian,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"[pickup, delivery]",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza],pizza,2.000000
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"[pickup, delivery]",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak],indpak,2.000000
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"[pickup, delivery]",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]","srilankan, indpak",2.000000
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]","italian, desserts, wine_bars",2.567669


## Finalización de la imputación y creación de columna de seguimiento

Con la imputación realizada, la columna `price_num` ya no contiene valores nulos que puedan afectar análisis futuros, y los valores faltantes se completaron de manera **inteligente**.  

Además, se crea una columna adicional que permite identificar cuáles valores fueron imputados, dejando un **registro claro** de las modificaciones realizadas en el DataFrame.  


In [101]:
# Crea una nueva columna 'price_num_imputado'.
# .isna() devuelve True para los valores nulos y False para los no nulos.
# Esta operación se realiza sobre la columna 'price' original.
df_res['price_num_imputado'] = df_res['price'].isna()

# Muestra el DataFrame actualizado para verificar que la nueva columna fue agregada correctamente.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num,price_num_imputado
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3.000000,False
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"[pickup, delivery]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3.000000,False
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"[pickup, delivery]",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]","cocktailbars, newamerican, speakeasies",2.000000,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2.000000,False
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,[delivery],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian],italian,3.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"[pickup, delivery]",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza],pizza,2.000000,False
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"[pickup, delivery]",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak],indpak,2.000000,True
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"[pickup, delivery]",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]","srilankan, indpak",2.000000,False
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]","italian, desserts, wine_bars",2.567669,True


## Cambio de tipo de variable en `price_num`

Una vez completada la imputación y verificación del DataFrame, se realiza el cambio de tipo de la columna `price_num` para que contenga únicamente **números enteros**.  

Este ajuste asegura que los valores de precio sean consistentes y estén listos para futuros análisis estadísticos o visualizaciones.  


In [102]:
# Redondea y convierte los valores en 'price_num' a enteros.
# 1. .round(): Redondea cada valor al entero más cercano. Por ejemplo, 2.7 se convierte en 3.
# 2. .astype(int): Convierte los valores redondeados a un tipo de dato entero.
df_res['price_num'] = df_res['price_num'].round().astype(int)

# Muestra el DataFrame actualizado para verificar que los cambios se hayan aplicado correctamente.
df_res

,id,alias,image_url,url,review_count,rating,transactions,price,phone,coordinates.latitude,coordinates.longitude,location.zip_code,address,aliases,aliases_2,price_num,price_num_imputado
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/ya6gjD...,https://www.yelp.com/biz/girl-and-the-goat-chi...,10509,4.4,[delivery],$$$,13124926262,41.884193,-87.647946,60607,809 W Randolph,"[newamerican, bars, bakeries]","newamerican, bars, bakeries",3,False
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/rHHvhR...,https://www.yelp.com/biz/the-purple-pig-chicag...,8854,4.3,"[pickup, delivery]",$$$,13124641744,41.890694,-87.624782,60611,444 N Michigan Ave,"[tapasmallplates, mediterranean, newamerican]","tapasmallplates, mediterranean, newamerican",3,False
2,gzhkdb6YoiFm5s3vriG1AA,gretel-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/-dxigk...,https://www.yelp.com/biz/gretel-chicago?adjust...,415,4.5,"[pickup, delivery]",$$,17737703427,41.917275,-87.698577,60647,2833 W Armitage Ave,"[cocktailbars, newamerican, speakeasies]","cocktailbars, newamerican, speakeasies",2,False
3,riT822EnU7y_5eCuJsd9sA,cindys-rooftop-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/XLdHyZ...,https://www.yelp.com/biz/cindys-rooftop-chicag...,2715,4.1,[delivery],$$,13127923502,41.881689,-87.625006,60603,12 S Michigan Ave,"[newamerican, seafood, breakfast_brunch]","newamerican, seafood, breakfast_brunch",2,False
4,GZsrGq6H8CQ4YlGtE_Bm0Q,ciccio-mio-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/eFIrEM...,https://www.yelp.com/biz/ciccio-mio-chicago-2?...,577,4.7,[delivery],$$$,13127963316,41.889390,-87.635240,60654,226 W Kinzie,[italian],italian,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Rn3o8GcYBPFJX5MWJn0r1Q,roberts-pizza-and-dough-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/0KtoeK...,https://www.yelp.com/biz/roberts-pizza-and-dou...,1106,4.4,"[pickup, delivery]",$$,13122651328,41.890570,-87.616451,60611,465 N McClurg Ct,[pizza],pizza,2,False
196,z6oM4x5zFSbPkXuGjhdXBg,dhanteraz-indian-fusion-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/d67HJw...,https://www.yelp.com/biz/dhanteraz-indian-fusi...,64,4.7,"[pickup, delivery]",NaN,17737998585,41.943438,-87.681687,60618,2116 W Roscoe St,[indpak],indpak,2,True
197,c6Mpi_TsS1iKrS1hyRsFxw,cafe-nova-chicago,https://s3-media0.fl.yelpcdn.com/bphoto/t2C795...,https://www.yelp.com/biz/cafe-nova-chicago?adj...,74,4.6,"[pickup, delivery]",$$,17736750736,41.999260,-87.660308,60626,6431 N Sheridan Rd,"[srilankan, indpak]","srilankan, indpak",2,False
198,6VbYWv_cm7AbBkz1hBlq_w,il-carciofo-chicago-2,https://s3-media0.fl.yelpcdn.com/bphoto/A226GE...,https://www.yelp.com/biz/il-carciofo-chicago-2...,106,3.9,[],NaN,18722745862,41.886454,-87.653983,60607,1045 W Fulton St,"[italian, desserts, wine_bars]","italian, desserts, wine_bars",3,True


### Eliminación de la Columna 'price'

En este paso, eliminamos la columna **'price'** del DataFrame ya que contamos con otra columna que proporciona la misma información pero de manera más completa y con valores numéricos. La eliminación de columnas redundantes es una práctica común en la limpieza de datos, ya que ayuda a reducir la complejidad y mejora el rendimiento del análisis. Esto también asegura que no haya inconsistencias o duplicaciones en la información que estamos utilizando para el análisis posterior.


In [103]:
# Elimina columna 'price'

df_res.drop(columns=['price'], inplace=True) # eliminamos la columna que ya no es necesaria

## Visualización del DataFrame final

Finalmente, se muestra la información del DataFrame ya completo y transformado.  

Esto permite **corroborar** que la limpieza, imputación y transformación de la columna `price_num` se realizaron correctamente, dejando un conjunto_


In [104]:
df_res.info() # Verificar que no haya valores nulos y los tipos de datos sean correctos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     200 non-null    object 
 1   alias                  200 non-null    object 
 2   image_url              200 non-null    object 
 3   url                    200 non-null    object 
 4   review_count           200 non-null    int64  
 5   rating                 200 non-null    float64
 6   transactions           200 non-null    object 
 7   phone                  200 non-null    object 
 8   coordinates.latitude   200 non-null    float64
 9   coordinates.longitude  200 non-null    float64
 10  location.zip_code      200 non-null    int64  
 11  address                200 non-null    object 
 12  aliases                200 non-null    object 
 13  aliases_2              200 non-null    object 
 14  price_num              200 non-null    int64  
 15  price_

### Ordenación de Columnas por Relevancia

En este paso, reorganizamos las columnas del DataFrame para colocar las más relevantes al principio. Esto facilita el análisis y mejora la visualización de los datos, permitiendo un acceso más rápido a las variables clave. Al mover las columnas más importantes al inicio, podemos centrarnos de manera más eficiente en las variables que influirán directamente en el análisis, y reducimos la complejidad al organizar la información de manera lógica y accesible.


In [105]:
# Ordenar las columnas 

columnas_ordenadas = [
    'id', 
    'alias', 
    'aliases',
    'price_num', 
    'price_num_imputado', 
    'phone', 
    'address', 
    'rating', 
    'review_count', 
    'transactions',  
    'image_url', 
    'url', 
    'location.zip_code', 
    'coordinates.longitude', 
    'coordinates.latitude' ] # Lista con el orden deseado de las columnas

df_res = df_res[columnas_ordenadas] # Reordenar las columnas del DataFrame según la lista definida

### Exportación del DataFrame Normalizado

Como último paso, exportamos el DataFrame que ha sido limpiado, organizado y normalizado, y que ahora está listo para su análisis. Esto nos permite guardar los datos procesados en un archivo CSV, que puede ser utilizado para análisis posteriores o compartido con otros miembros del equipo. La exportación es crucial para mantener un registro de los datos en su forma final, facilitando el trabajo futuro y la reproducibilidad del análisis.

El archivo exportado contendrá todas las modificaciones realizadas, incluidas las eliminaciones, reorganizaciones de columnas y normalización de los datos.


In [106]:
# exportar el DataFrame final a un archivo CSV

df_res.to_csv('Datos_YELP_limpios.csv', index=False) # Exportar el DataFrame limpio a un archivo CSV sin incluir el índice